In [242]:
import io
import sys
import copy

def load_data(file_name):
    data = {}
    with io.open(file_name) as f:
        books, libraries, days = f.readline()[:-1].split(" ")
        data["books_number"] = int(books)
        data["librs_number"] = int(libraries)
        data["days"] = int(days)
        data["books_scores"] = {idx: int(val) for idx, val in enumerate(f.readline()[:-1].split(" "))}
        data["librs"] = {}
        for lib in range(int(libraries)):
            b, s, sh = f.readline()[:-1].split(" ")
            data["librs"][lib] = {}
            data["librs"][lib]["books"] = int(b)
            data["librs"][lib]["signup"] = int(s)
            data["librs"][lib]["ship"] = int(sh)
            data["librs"][lib]["books_ids"] = [int(b) for b in f.readline()[:-1].split(" ")]
            temp_to_sort = [(idx, data["books_scores"][idx]) for idx in data["librs"][lib]["books_ids"]]
            data["librs"][lib]["sorted_books_ids"] = [j[0] for j in sorted(temp_to_sort, key=lambda x: x[1], reverse=True)]
            data["librs"][lib]["total_score"] = sum([data["books_scores"][i] for i in data["librs"][lib]["books_ids"]])
    
    return data

In [243]:
def process(data, out_file, flag0=True):
    libs = data["librs"]
    sorted_libs = {k: v for k, v in sorted(libs.items(), key=lambda item: item[1]["total_score"], reverse=True)}
    # sorted_libs = {k: v for k, v in sorted(libs.items(), 
    #                key=lambda item: 0.4 * item[1]["total_score"] + 0.3 * item[1]["ship"] + 0.2 * len(item[1]["books_ids"]) + 0.1 * item[1]["signup"],
    #                reverse=True)
    #                }
    sent = set()
    for l in sorted_libs:
        score = 0
        duplicate_books_ids = []        
        for book in sorted_libs[l]["sorted_books_ids"]:
            if book in sent:
                score += data["books_scores"][book]
                duplicate_books_ids.append(book)
                continue
            sent.add(book)
        sorted_libs[l]["duplicates_scores"] = sorted_libs[l]["total_score"] - score
        sorted_libs[l]["new_sorted_books_ids"] = list(set(sorted_libs[l]["sorted_books_ids"]) - set(duplicate_books_ids))

    # sorted_libs = {k: v for k, v in sorted(libs.items(), key=lambda item: item[1]["ship"], reverse=True)}
    sorted_libs = {k: v for k, v in sorted(libs.items(), 
                   key=lambda item: len(item[1]["new_sorted_books_ids"]),
                   reverse=True)
                   }
    days = data["days"]
    count_libs = 0
    output_dict = {}
    already_shipped = []
    for lib in sorted_libs:
        if days <= 0:
           break
        output_dict[lib] = {}        
        books_window = days - sorted_libs[lib]["signup"]
        books_per_day = sorted_libs[lib]["ship"]
        shiped_books = []
        to_ship = sorted_libs[lib]["new_sorted_books_ids"]
        for remaining_day in range(books_window):
            for per_day in range(books_per_day):
                if not to_ship:
                    break
                curr_book = to_ship.pop(0)
                shiped_books.append(curr_book)
        output_dict[lib]["books"] = shiped_books
        output_dict[lib]["len"] = len(shiped_books)
        count_libs += 1
        days -= sorted_libs[lib]["signup"]
    
    with open(out_file, "w") as wf:
        if flag0:
            count_libs -= 1
        count_libs = sum([1 for l in output_dict if output_dict[l]["books"]])
        wf.write(str(count_libs) + "\n")
        c = 0
        for l in output_dict:
            if output_dict[l]["books"]:
                wf.write(str(l) + " " + str(output_dict[l]["len"]) + "\n")
                for shiped in output_dict[l]["books"]:
                    wf.write(str(shiped) + " ")
                c += 1
                if c < len(output_dict.keys()):
                    wf.write("\n")

In [244]:
data_a = load_data("a_example.txt")
print(data_a)
process(data_a, "a_example.out", flag0=False)


{'books_number': 6, 'librs_number': 2, 'days': 7, 'books_scores': {0: 1, 1: 2, 2: 3, 3: 6, 4: 5, 5: 4}, 'librs': {0: {'books': 5, 'signup': 2, 'ship': 2, 'books_ids': [0, 1, 2, 3, 4], 'sorted_books_ids': [3, 4, 2, 1, 0], 'total_score': 17}, 1: {'books': 4, 'signup': 3, 'ship': 1, 'books_ids': [0, 2, 3, 5], 'sorted_books_ids': [3, 5, 2, 0], 'total_score': 14}}}


In [245]:
data_b = load_data("b_read_on.txt")
# print(data_b)
process(data_b, "b_read_on.out")

In [246]:
data_c = load_data("c_incunabula.txt")
# print(data_c)
process(data_c, "c_incunabula.out")

In [247]:
data_e = load_data("e_so_many_books.txt")
# print(data_e)
process(data_e, "e_so_many_books.out")

In [248]:
data_f = load_data("f_libraries_of_the_world.txt")
# print(data_f)
process(data_f, "f_libraries_of_the_world.out")

In [249]:
data_d = load_data("d_tough_choices.txt")
# print(data_d)
process(data_d, "d_tough_choices.out")